## Dependencias

In [246]:
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 500

# Internet fijo

## Fuentes

### Accesos de Internet Fijo desde 2017-2T

desde [postdata](https://postdata.gov.co/dataset/suscriptores-e-ingresos-de-internet-fijo/resource/540ea080-bf16-4d63-911f-3b4814e8e4f1#%7Bview-grid:%7BcolumnsWidth:%5B%7Bcolumn:!accesos,width:108%7D%5D%7D%7D)

In [2]:
internet_fijo = pd.read_csv('data/sources/internetfijo.csv', sep=';')

In [3]:
internet_fijo.shape

(1660574, 15)

In [86]:
internet_fijo.sample(1)

,ANNO,TRIMESTRE,ID_EMPRESA,EMPRESA,ID_MUNICIPIO,MUNICIPIO,ID_DEPARTAMENTO,DEPARTAMENTO,ID_SEGMENTO,SEGMENTO,VELOCIDAD_EFECTIVA_DOWNSTREAM,VELOCIDAD_EFECTIVA_UPSTREAM,ID_TECNOLOGIA,TECNOLOGIA,ACCESOS
37556,2017,2,900548102,AZTECA COMUNICACIONES COLOMBIA S.A.S,52110,BUESACO,52,NARIÑO,107,Corporativo,25,"12,5",108,Fiber to the home (FTTH),3


## Ideas

### Número de accesos de internet fijo por trimestre

In [148]:
fijo_accesos = internet_fijo.groupby(['ANNO', 'TRIMESTRE']).ACCESOS.sum()

In [281]:
fijo_accesos

ANNO  TRIMESTRE
2017  2            6193671
      3            6284505
      4            6352946
2018  1            6479592
      2            6578663
      3            6660066
      4            6731509
2019  1            6800865
      2            6911078
      3            7012936
      4            6982443
2020  1            7165355
      2            7490454
      3            7698521
      4            7835342
2021  1            8127334
      2            8292381
      3            8341313
      4            8444874
2022  1            8546208
      2            8482581
Name: ACCESOS, dtype: int64

In [149]:
fijo_accesos.to_csv('data/internetfijo_accesos.csv')

### Crecimiento trimestral en accesos de internet fijo

In [304]:
fijo_accesos_crecimiento = pd.concat([
    fijo_accesos.diff(),
    fijo_accesos.sub(fijo_accesos.iloc[0]),
    fijo_accesos.div(fijo_accesos.shift(1)),
    fijo_accesos.div(fijo_accesos.iloc[0])
], axis=1)

In [306]:
fijo_accesos_crecimiento.columns = ['diferencia trimestral', 'diferencia respecto a 2017-2T', 'crecimiento trimestral', 'crecimiento respecto a 2017-2T']

In [307]:
fijo_accesos_crecimiento

diferencia trimestral  diferencia respecto a 2017-2T  \
ANNO TRIMESTRE                                                         
2017 2                            NaN                              0   
     3                        90834.0                          90834   
     4                        68441.0                         159275   
2018 1                       126646.0                         285921   
     2                        99071.0                         384992   
     3                        81403.0                         466395   
     4                        71443.0                         537838   
2019 1                        69356.0                         607194   
     2                       110213.0                         717407   
     3                       101858.0                         819265   
     4                       -30493.0                         788772   
2020 1                       182912.0                         971684   
     2                       325099.0                        1296783   
     3                       208067.0                        1504850   
     4                       136821.0                        1641671   
2021 1                       291992.0                        1933663   
     2                       165047.0                        2098710   
     3                        48932.0                        2147642   
     4                       103561.0                        2251203   
2022 1                       101334.0                        2352537   
     2                       -63627.0                        2288910   

                crecimiento trimestral  crecimiento respecto a 2017-2T  
ANNO TRIMESTRE                                                          
2017 2                             NaN                        1.000000  
     3                        1.014666                        1.014666  
     4                        1.010890                        1.025716  
2018 1                        1.019935                        1.046163  
     2                        1.015290                        1.062159  
     3                        1.012374                        1.075302  
     4                        1.010727                        1.086837  
2019 1                        1.010303                        1.098035  
     2                        1.016206                        1.115829  
     3                        1.014738                        1.132275  
     4                        0.995652                        1.127351  
2020 1                        1.026196                        1.156883  
     2                        1.045371                        1.209372  
     3                        1.027778                        1.242966  
     4                        1.017772                        1.265056  
2021 1                        1.037266                        1.312200  
     2                        1.020308                        1.338848  
     3                        1.005901                        1.346748  
     4                        1.012415                        1.363468  
2022 1                        1.011999                        1.379829  
     2                        0.992555                        1.369556

In [308]:
fijo_accesos_crecimiento.to_csv('data/internetfijo_accesos_crecimento.csv')

Número de accesos por departamento

In [171]:
fijo_departamentos = internet_fijo.groupby(['ANNO', 'TRIMESTRE', 'ID_DEPARTAMENTO']).ACCESOS.sum().reset_index().pivot_table(index=['ANNO', 'TRIMESTRE'], columns='ID_DEPARTAMENTO', values='ACCESOS').fillna(0).astype(int)

In [175]:
depto_dict = internet_fijo.groupby('ID_DEPARTAMENTO').DEPARTAMENTO.first().to_dict()

In [179]:
fijo_departamentos = fijo_departamentos.rename(columns=depto_dict)

In [235]:
fijo_departamentos

ID_DEPARTAMENTO  COLOMBIA  ANTIOQUIA  ATLÁNTICO  BOGOTÁ D.C.  BOLÍVAR  BOYACÁ  \
ANNO TRIMESTRE                                                                  
2017 2                 52    1074066     314991      1776759   174788  102904   
     3                 90    1090819     322437      1807827   174669  103228   
     4                  0    1098843     328389      1818094   177184  105391   
2018 1                  0    1113650     335464      1826350   180168  108764   
     2                  0    1127759     338205      1857309   183550  111668   
     3                  0    1149057     343197      1875396   183812  114666   
     4                  0    1164147     346822      1876271   186967  116210   
2019 1                  0    1187759     329666      1874506   191419  120297   
     2                  0    1198415     343938      1890765   194076  121760   
     3                  0    1212047     351804      1915409   198475  124919   
     4                  0    1232910     354675      1916910   185788  116349   
2020 1                  0    1258347     362839      1956766   191306  120739   
     2                  0    1322906     376574      2024199   197383  125176   
     3                  0    1346629     385913      2068167   198025  132694   
     4                  0    1363693     391057      2087437   197095  137564   
2021 1                  0    1407864     407995      2137106   207639  147376   
     2                  0    1464234     421947      2163410   221551  151121   
     3                  0    1489033     433582      2152436   229480  153930   
     4                  0    1509713     425338      2169165   234193  154752   
2022 1                  0    1529908     436012      2189547   236081  156989   
     2                  0    1524474     424788      2197597   227879  157824   

ID_DEPARTAMENTO  CALDAS  CAQUETA   CAUCA   CESAR  ...  VALLE DEL CAUCA  \
ANNO TRIMESTRE                                    ...                    
2017 2           117090    20312   63681   76233  ...           685493   
     3           118765    22299   65006   77755  ...           698081   
     4           119291    19463   65547   79562  ...           707051   
2018 1           120931    19190   69939   82677  ...           719755   
     2           122103    20378   70370   84115  ...           731687   
     3           121484    21443   75890   87011  ...           741593   
     4           123072    22595   75879   88889  ...           749987   
2019 1           125136    22874   75130   91285  ...           763692   
     2           127633    22304   80819   99853  ...           768930   
     3           127296    26741   76234  101159  ...           779626   
     4           121706    26931   75525  102363  ...           782509   
2020 1           129564    28707   77510  102930  ...           796990   
     2           137568    30875   87682  109985  ...           827014   
     3           142560    31998   83037  111163  ...           850067   
     4           152813    28982   84165  113166  ...           864036   
2021 1           155484    33684  100855  122146  ...           879885   
     2           157807    33448  104264  122143  ...           885944   
     3           155630    35749   96634  128378  ...           886623   
     4           155106    35620   92899  119452  ...           892882   
2022 1           155579    32083   93854  128924  ...           889558   
     2           162890    29840   95361  121487  ...           907437   

ID_DEPARTAMENTO  ARAUCA  CASANARE  PUTUMAYO  \
ANNO TRIMESTRE                                
2017 2             9242     32173     10897   
     3             9580     32855     11435   
     4            10647     33843     12200   
2018 1            11105     34136     12772   
     2            11263     34762     13534   
     3            11260     34124     14645   
     4            11272     33711     15553   
2019 1    

In [181]:
fijo_departamentos.to_csv('data/internetfijo_accesos_por_departamento.csv')

Crecimiento por departamento

In [239]:
def get_crecimiento(tabla):
    
    tabla_c = pd.concat([tabla.loc[(2017, 2)], tabla.loc[(2022, 2)]], axis=1).sort_values((2022, 2), ascending=False)
    tabla_c.columns = ['2017', '2022']
    tabla_c = pd.concat([tabla_c, (tabla_c['2022'] / tabla_c['2017']).rename('crecimiento')], axis=1)
    tabla_c = tabla_c.sort_values('crecimiento', ascending=False)
    return tabla_c

In [240]:
fijo_departamentos_c = get_crecimiento(fijo_departamentos)

In [241]:
fijo_departamentos_c

,2017,2022,crecimiento
ID_DEPARTAMENTO,,,
GUAINÍA,49,111,2.265306
ARAUCA,9242,20721,2.242047
AMAZONAS,422,770,1.824645
CUNDINAMARCA,334699,573672,1.713994
MAGDALENA,87350,142222,1.628185
CESAR,76233,121487,1.593627
HUILA,94906,149362,1.573789
LA GUAJIRA,33598,52868,1.573546
CASANARE,32173,50109,1.557486


In [234]:
fijo_departamentos_c.to_csv('data/internetfijo_accesos_por_departamento_crecimiento.csv')

Concentración entre departamentos

In [270]:
def get_concentracion(tabla):
    
    tabla_p = tabla.div(tabla.sum(axis=1), axis=0)

    tabla_p = pd.concat([
        tabla_p.loc[(2017, 2)].sort_values(ascending=False),
        tabla_p.loc[(2022, 2)].sort_values(ascending=False)], axis=1)

    tabla_p.columns = ['porcentaje_2017', 'porcentaje_2022']
    tabla_p['porcentaje_diferencia'] = (tabla_p.porcentaje_2022 - tabla_p.porcentaje_2017)
    tabla_p[['suma_cumulativa_2017', 'suma_cumulativa_2022']] = tabla_p[['porcentaje_2017', 'porcentaje_2022']].cumsum()
    
    return tabla_p

In [271]:
fijo_departamentos_p = get_concentracion(fijo_departamentos)

In [272]:
fijo_departamentos_p

,porcentaje_2017,porcentaje_2022,porcentaje_diferencia,suma_cumulativa_2017,suma_cumulativa_2022
ID_DEPARTAMENTO,,,,,
BOGOTÁ D.C.,0.286867,0.259072,-0.027795,0.286867,0.259072
ANTIOQUIA,0.173413,0.179718,0.006305,0.460280,0.438790
VALLE DEL CAUCA,0.110676,0.106977,-0.003700,0.570957,0.545766
CUNDINAMARCA,0.054039,0.067629,0.013591,0.624996,0.613396
SANTANDER,0.052773,0.048803,-0.003970,0.677768,0.662199
ATLÁNTICO,0.050857,0.050078,-0.000779,0.728625,0.712276
BOLÍVAR,0.028220,0.026864,-0.001356,0.756846,0.739141
TOLIMA,0.023878,0.023927,0.000048,0.780724,0.763067
RISARALDA,0.023514,0.026170,0.002656,0.804238,0.789237


In [216]:
fijo_departamentos_p.to_csv('data/internetfijo_accesos_por_departamento_concentracion.csv')

Número de accesos por empresa

In [259]:
fijo_por_empresa = internet_fijo.groupby(['ID_EMPRESA', 'ANNO', 'TRIMESTRE']).ACCESOS.sum().reset_index().pivot_table(index=['ANNO', 'TRIMESTRE'], columns='ID_EMPRESA', values='ACCESOS').fillna(0).astype(int)

In [260]:
emp_dict = internet_fijo.groupby('ID_EMPRESA').EMPRESA.first().to_dict()

In [263]:
fijo_por_empresa = fijo_por_empresa.rename(columns=emp_dict)

In [264]:
fijo_por_empresa = fijo_por_empresa[fijo_por_empresa.sum().sort_values(ascending=False).index]

In [265]:
fijo_por_empresa

ID_EMPRESA      COMUNICACION CELULAR S A COMCEL S A  \
ANNO TRIMESTRE                                        
2017 2                                            0   
     3                                            0   
     4                                            0   
2018 1                                            0   
     2                                            0   
     3                                            0   
     4                                            0   
2019 1                                            0   
     2                                      2610782   
     3                                      2647664   
     4                                      2664110   
2020 1                                      2747552   
     2                                      2879580   
     3                                      3005507   
     4                                      3065976   
2021 1                                      3175493   
     2                                      3209411   
     3                                      3180321   
     4                                      3231312   
2022 1                                      3250531   
     2                                      3242542   

ID_EMPRESA      UNE EPM TELECOMUNICACIONES S.A.  \
ANNO TRIMESTRE                                    
2017 2                                  1250018   
     3                                  1275003   
     4                                  1278451   
2018 1                                  1311196   
     2                                  1325817   
     3                                  1354791   
     4                                  1382814   
2019 1                                  1415373   
     2                                  1442224   
     3                                  1474656   
     4                                  1506821   
2020 1                                  1517996   
     2                                  1594990   
     3                                  1594920   
     4                                  1618325   
2021 1                                  1657301   
     2                                  1686313   
     3                                  1714644   
     4                                  1733882   
2022 1                                  1762784   
     2                                  1776438   

ID_EMPRESA      COLOMBIA TELECOMUNICACIONES S.A. E.S.P.  TELMEX COLOMBIA S.A.  \
ANNO TRIMESTRE                                                                  
2017 2                                           966600               2278348   
     3                                           968908               2313890   
     4                                           967879               2349281   
2018 1                                           986358               2384831   
     2                                          1001954               2423948   
     3                                          1008300               2465033   
     4                                          1002199               2499415   
2019 1                                           997198               2562264   
     2                                           984774                     0   
     3                                           992369                     0   
     4                                           994351                     0   
2020 1                                          1005305                     0   
     2                                          1184090                     0   
     3                                          1172659                     0   
     4                                          1156342                     0   
2021 1                                          1168215                     0   
     2                                          1165912                     0   
     3               

In [269]:
fijo_por_empresa['RURALINK S.A.S']

ANNO  TRIMESTRE
2017  2                0
      3                0
      4                0
2018  1                0
      2                0
      3                0
      4                0
2019  1                0
      2                0
      3                0
      4                0
2020  1                0
      2            14797
      3            19489
      4            23286
2021  1            26190
      2            31470
      3            34332
      4            36294
2022  1            41140
      2            41083
Name: RURALINK S.A.S, dtype: int64

In [142]:
fijo_por_empresa.to_csv('data/internetfijo_accesos_por_empresa.csv')

31% de las empresas en el registro no tienen ningún acceso durante el último trimestre. Sólo 2% tienen más de 10 mil.

In [38]:
pd.cut(fijo_por_empresa.iloc[-1], bins=[-1, 0, 1e2, 1e3, 1e4, 1e10]).value_counts(normalize=True).sort_index()

(-1.0, 0.0]                 0.311909
(0.0, 100.0]                0.158790
(100.0, 1000.0]             0.347826
(1000.0, 10000.0]           0.155009
(10000.0, 10000000000.0]    0.026465
Name: (2022, 2), dtype: float64

In [275]:
fijo_empresa_c = get_crecimiento(fijo_por_empresa)

In [276]:
fijo_empresa_c.sort_values('2017', ascending=False).head(20)

,2017,2022,crecimiento
ID_EMPRESA,,,
TELMEX COLOMBIA S.A.,2278348,0,0.000000
UNE EPM TELECOMUNICACIONES S.A.,1250018,1776438,1.421130
COLOMBIA TELECOMUNICACIONES S.A. E.S.P.,966600,1230778,1.273306
EMPRESA DE TELECOMUNICACIONES DE BOGOTA S.A. ESP,643572,684980,1.064341
EDATEL S.A.,201142,152522,0.758280
AZTECA COMUNICACIONES COLOMBIA S.A.S,166025,18226,0.109779
DIRECTV COLOMBIA LTDA,126501,158141,1.250117
EMPRESA DE TELECOMUNICACIONES DE BUCARAMANGA S.A. E.S.P.,125980,0,0.000000
METROTEL S.A,120576,0,0.000000


In [277]:
fijo_empresa_c.sort_values('2022', ascending=False).head(20)

,2017,2022,crecimiento
ID_EMPRESA,,,
COMUNICACION CELULAR S A COMCEL S A,0,3242542,inf
UNE EPM TELECOMUNICACIONES S.A.,1250018,1776438,1.421130
COLOMBIA TELECOMUNICACIONES S.A. E.S.P.,966600,1230778,1.273306
EMPRESA DE TELECOMUNICACIONES DE BOGOTA S.A. ESP,643572,684980,1.064341
DIRECTV COLOMBIA LTDA,126501,158141,1.250117
EDATEL S.A.,201142,152522,0.758280
H V TELEVISION S.A.S.,37284,107767,2.890436
EMPRESAS MUNICIPALES DE CALI EICE E.S.P,106431,100706,0.946209
DIALNET DE COLOMBIA S.A ESP.,13447,60674,4.512084


In [278]:
fijo_empresa_c[(fijo_empresa_c.crecimiento < np.inf) & fijo_empresa_c['2017'] > 0]

,2017,2022,crecimiento
ID_EMPRESA,,,
BARAK TECNOLOGIA INFORMACION Y COMUNICACIONES SAS,1,2461,2461.000000
INTERNET Y TELECOMUNICACIONES DE COLOMBIA S.A.S.,1,1268,1268.000000
MEDIAWIRE TELECOMUNICACIONES S.A.S,1,955,955.000000
CABLEMAS S.A.S,39,17427,446.846154
LEGON TELECOMUNICACIONES S.A.S.,151,57560,381.192053
...,...,...,...
AIRMAX.NET TELECOMUNICACIONES SAS,191,0,0.000000
SISTELEC LTDA,33,0,0.000000
EMPRESA DE TELECOMUNICACIONES DE LA ORINOQUIA S.A. E.S.P.,573,0,0.000000


In [279]:
fijo_empresa_p = get_concentracion(fijo_por_empresa)

In [280]:
fijo_empresa_p

,porcentaje_2017,porcentaje_2022,porcentaje_diferencia,suma_cumulativa_2017,suma_cumulativa_2022
ID_EMPRESA,,,,,
TELMEX COLOMBIA S.A.,0.367851,0.000000,-0.367851,0.367851,0.000000
UNE EPM TELECOMUNICACIONES S.A.,0.201822,0.209422,0.007600,0.569673,0.209422
COLOMBIA TELECOMUNICACIONES S.A. E.S.P.,0.156063,0.145095,-0.010968,0.725735,0.354517
EMPRESA DE TELECOMUNICACIONES DE BOGOTA S.A. ESP,0.103908,0.080751,-0.023157,0.829643,0.435268
EDATEL S.A.,0.032475,0.017981,-0.014495,0.862119,0.453249
...,...,...,...,...,...
AJFIBER.NET S.A.S.,0.000000,0.000056,0.000056,1.000000,0.999934
REMG INGENIERIA SAS,0.000000,0.000017,0.000017,1.000000,0.999950
COMSISTELCO S.A.S.,0.000000,0.000000,0.000000,1.000000,0.999950


Las primeras 3 empresas tienen 3/4 de los accesos durante el último trimestre

In [69]:
fijo_por_empresa.iloc[-1][(fijo_por_empresa.iloc[-1].sort_values(ascending=False) / fijo_por_empresa.iloc[-1].sum()).cumsum() <= .75]

EMPRESA
COMUNICACION CELULAR S A COMCEL S A        3242542
UNE EPM TELECOMUNICACIONES S.A.            1776438
COLOMBIA TELECOMUNICACIONES S.A. E.S.P.    1230778
Name: (2022, 2), dtype: int64

Y las primeras 11 tienen el 90%

In [70]:
fijo_por_empresa.iloc[-1][(fijo_por_empresa.iloc[-1].sort_values(ascending=False) / fijo_por_empresa.iloc[-1].sum()).cumsum() <= .9]

EMPRESA
COMUNICACION CELULAR S A COMCEL S A                 3242542
UNE EPM TELECOMUNICACIONES S.A.                     1776438
COLOMBIA TELECOMUNICACIONES S.A. E.S.P.             1230778
EMPRESA DE TELECOMUNICACIONES DE BOGOTA S.A. ESP     684980
EDATEL S.A.                                          152522
DIRECTV COLOMBIA LTDA                                158141
EMPRESAS MUNICIPALES DE CALI EICE E.S.P              100706
H V TELEVISION S.A.S.                                107767
DIALNET DE COLOMBIA S.A   ESP.                        60674
LEGON TELECOMUNICACIONES S.A.S.                       57560
RURALINK S.A.S                                        41083
Name: (2022, 2), dtype: int64

La serie de tiempo de accesos para estas 11 empresas

In [65]:
fijo_por_empresa[fijo_por_empresa.iloc[-1][((fijo_por_empresa.iloc[-1].sort_values(ascending=False) / fijo_por_empresa.iloc[-1].sum()).cumsum() <= .9)].index]

EMPRESA         COMUNICACION CELULAR S A COMCEL S A  \
ANNO TRIMESTRE                                        
2017 2                                            0   
     3                                            0   
     4                                            0   
2018 1                                            0   
     2                                            0   
     3                                            0   
     4                                            0   
2019 1                                            0   
     2                                      2610782   
     3                                      2647664   
     4                                      2664110   
2020 1                                      2747552   
     2                                      2879580   
     3                                      3005507   
     4                                      3065976   
2021 1                                      3175493   
     2                                      3209411   
     3                                      3180321   
     4                                      3231312   
2022 1                                      3250531   
     2                                      3242542   

EMPRESA         UNE EPM TELECOMUNICACIONES S.A.  \
ANNO TRIMESTRE                                    
2017 2                                  1250018   
     3                                  1275003   
     4                                  1278451   
2018 1                                  1311196   
     2                                  1325817   
     3                                  1354791   
     4                                  1382814   
2019 1                                  1415373   
     2                                  1442224   
     3                                  1474656   
     4                                  1506821   
2020 1                                  1517996   
     2                                  1594990   
     3                                  1594920   
     4                                  1618325   
2021 1                                  1657301   
     2                                  1686313   
     3                                  1714644   
     4                                  1733882   
2022 1                                  1762784   
     2                                  1776438   

EMPRESA         COLOMBIA TELECOMUNICACIONES S.A. E.S.P.  \
ANNO TRIMESTRE                                            
2017 2                                           966600   
     3                                           968908   
     4                                           967879   
2018 1                                           986358   
     2                                          1001954   
     3                                          1008300   
     4                                          1002199   
2019 1                                           997198   
     2                                           984774   
     3                                           992369   
     4                                           994351   
2020 1                                          1005305   
     2                                          1184090   
     3                                          1172659   
     4                                          1156342   
2021 1                                          1168215   
     2                                          1165912   
     3                                          1170972   
     4                                          1183775   
2022 1                                          1207659   
     2                                          1230778   

EMPRESA         EMPRESA DE TELECOMUNICACIONES DE BOGOTA S.A. ESP  EDATEL S.A.  \
ANNO TRIMESTRE                                                                  
2017 2                                  

In [143]:
fijo_por_empresa[fijo_por_empresa.iloc[-1][((fijo_por_empresa.iloc[-1].sort_values(ascending=False) / fijo_por_empresa.iloc[-1].sum()).cumsum() <= .9)].index].to_csv('data/internetfijo_accesos_por_empresa_top11.csv')

Número de accesos por segmento

In [136]:
fijo_segmentos = internet_fijo.groupby(['ANNO', 'TRIMESTRE', 'SEGMENTO']).ACCESOS.sum().reset_index().pivot_table(index=['ANNO', 'TRIMESTRE'], columns='SEGMENTO', values='ACCESOS').fillna(0).astype(int)

In [137]:
fijo_segmentos

SEGMENTO        Corporativo  Residencial - Estrato 1  Residencial - Estrato 2  \
ANNO TRIMESTRE                                                                  
2017 2               543301                   627568                  2150384   
     3               548500                   618579                  2194890   
     4               553361                   687170                  2164659   
2018 1               552284                   637444                  2283184   
     2               557968                   645264                  2335155   
     3               564914                   649705                  2370713   
     4               586663                   668447                  2348570   
2019 1               565426                   685221                  2426176   
     2               572870                   738404                  2443491   
     3               582782                   754212                  2465173   
     4               588049                   666655                  2462577   
2020 1               591089                   707079                  2521295   
     2               576624                   760537                  2707218   
     3               557203                   816828                  2829020   
     4               564238                   852702                  2869772   
2021 1               573557                   907113                  2994041   
     2               589992                   984806                  3020039   
     3               591077                   977590                  3026424   
     4               625753                   977291                  3045757   
2022 1               637775                  1021359                  3015840   
     2               644795                   921464                  2987883   

SEGMENTO        Residencial - Estrato 3  Residencial - Estrato 4  \
ANNO TRIMESTRE                                                     
2017 2                          1769908                   677615   
     3                          1786154                   704760   
     4                          1796065                   711083   
2018 1                          1816161                   731824   
     2                          1821173                   743028   
     3                          1843357                   762464   
     4                          1854413                   778737   
2019 1                          1868931                   792458   
     2                          1880154                   798939   
     3                          1919776                   815037   
     4                          1955641                   831580   
2020 1                          2010906                   845055   
     2                          2085112                   860271   
     3                          2131646                   857726   
     4                          2165894                   863603   
2021 1                          2242305                   873832   
     2                          2264483                   874735   
     3                          2269356                   875659   
     4                          2306577                   883244   
2022 1                          2340604                   904178   
     2                          2402403                   910207   

SEGMENTO        Residencial - Estrato 5  Residencial - Estrato 6  \
ANNO TRIMESTRE                                                     
2017 2                           228633                   167010   
     3                           228289                   168601   
     4                           232036                   169578   
2018 1                           240024                   171966   
     2                           243523                   174089   
     3                           245494                   177098   
     4                     

In [144]:
fijo_segmentos.to_csv('data/internetfijo_accesos_por_segmento.csv')

Porcentaje de accesos por segmento

In [140]:
fijo_segmentos.div(fijo_segmentos.sum(axis=1), axis=0) * 100

SEGMENTO        Corporativo  Residencial - Estrato 1  Residencial - Estrato 2  \
ANNO TRIMESTRE                                                                  
2017 2             8.771874                10.132408                34.719054   
     3             8.727815                 9.842923                34.925424   
     4             8.710305                10.816557                34.073310   
2018 1             8.523438                 9.837718                35.236540   
     2             8.481480                 9.808437                35.495890   
     3             8.482108                 9.755234                35.595939   
     4             8.715178                 9.930121                34.889205   
2019 1             8.314031                10.075498                35.674521   
     2             8.289155                10.684353                35.356148   
     3             8.310100                10.754583                35.151797   
     4             8.421823                 9.547590                35.268129   
2020 1             8.249263                 9.868025                35.187301   
     2             7.698118                10.153417                36.142242   
     3             7.237793                10.610194                36.747578   
     4             7.201192                10.882767                36.625995   
2021 1             7.057136                11.161261                36.839153   
     2             7.114868                11.876034                36.419443   
     3             7.086139                11.719858                36.282345   
     4             7.409856                11.572594                36.066340   
2022 1             7.462666                11.951020                35.288633   
     2             7.601401                10.863014                35.223749   

SEGMENTO        Residencial - Estrato 3  Residencial - Estrato 4  \
ANNO TRIMESTRE                                                     
2017 2                        28.576074                10.940442   
     3                        28.421554                11.214248   
     4                        28.271372                11.192965   
2018 1                        28.028941                11.294291   
     2                        27.683026                11.294514   
     3                        27.677759                11.448295   
     4                        27.548251                11.568535   
2019 1                        27.480784                11.652312   
     2                        27.204931                11.560266   
     3                        27.374783                11.621908   
     4                        28.007977                11.909585   
2020 1                        28.064290                11.793624   
     2                        27.836924                11.484898   
     3                        27.689033                11.141439   
     4                        27.642622                11.021893   
2021 1                        27.589675                10.751767   
     2                        27.307995                10.548659   
     3                        27.206220                10.497856   
     4                        27.313338                10.458936   
2022 1                        27.387632                10.579874   
     2                        28.321604                10.730307   

SEGMENTO        Residencial - Estrato 5  Residencial - Estrato 6  \
ANNO TRIMESTRE                                                     
2017 2                         3.691397                 2.696462   
     3                         3.632569                 2.682805   
     4                         3.652416                 2.669281   
2018 1                         3.704307                 2.653963   
     2                         3.701710                 2.646267   
     3                         3.686060                 2.659103   
     4                     

In [145]:
fijo_segmentos.div(fijo_segmentos.sum(axis=1), axis=0).to_csv('data/internetfijo_accesos_por_segmento_porcentaje.csv')

Número de accesos residenciales según estrato socioeconómico

In [87]:
fijo_residencial_por_segmento = internet_fijo[internet_fijo.SEGMENTO.str.contains('Residencial|Sin')][['ANNO', 'TRIMESTRE', 'SEGMENTO', 'ACCESOS']]

In [90]:
fijo_residencial_por_segmento.SEGMENTO = fijo_residencial_por_segmento.SEGMENTO.apply(lambda x: x[-1] if 'Residencial' in x else 0).astype(int)

In [91]:
fijo_residencial_por_segmento = fijo_residencial_por_segmento.groupby(['ANNO', 'TRIMESTRE', 'SEGMENTO']).ACCESOS.sum().reset_index().pivot_table(index=['ANNO', 'TRIMESTRE'], columns='SEGMENTO', values='ACCESOS')

In [92]:
fijo_residencial_por_segmento

SEGMENTO             0        1        2        3       4       5       6
ANNO TRIMESTRE                                                           
2017 2           29252   627568  2150384  1769908  677615  228633  167010
     3           34732   618579  2194890  1786154  704760  228289  168601
     4           38994   687170  2164659  1796065  711083  232036  169578
2018 1           46705   637444  2283184  1816161  731824  240024  171966
     2           58463   645264  2335155  1821173  743028  243523  174089
     3           46321   649705  2370713  1843357  762464  245494  177098
     4           53532   668447  2348570  1854413  778737  255108  186039
2019 1           30644   685221  2426176  1868931  792458  246958  185051
     2           48497   738404  2443491  1880154  798939  246541  182182
     3           39816   754212  2465173  1919776  815037  249259  186881
     4           40800   666655  2462577  1955641  831580  249125  188016
2020 1           48946   707079  2521295  2010906  845055  249547  191438
     2           57374   760537  2707218  2085112  860271  250953  192365
     3           59118   816828  2829020  2131646  857726  252530  194450
     4           65211   852702  2869772  2165894  863603  255215  198707
2021 1           70466   907113  2994041  2242305  873832  261438  204582
     2           88251   984806  3020039  2264483  874735  263371  206704
     3          119691   977590  3026424  2269356  875659  264698  216818
     4          122121   977291  3045757  2306577  883244  268936  215195
2022 1          136175  1021359  3015840  2340604  904178  274351  215926
     2          121454   921464  2987883  2402403  910207  278635  212165

Sería útil comparar estos valores con el número de hogares según estrato o el número de personas que habitan hogares por estrato para tener una idea del nivel de acceso a telecomunicaciones. No encuentro este dato.

Velocidad efectiva media de bajada y subida por trimestre

In [129]:
fijo_velocidad_down = internet_fijo.groupby(['ANNO', 'TRIMESTRE'])['VELOCIDAD_EFECTIVA_DOWNSTREAM']
fijo_velocidad_up = internet_fijo.groupby(['ANNO', 'TRIMESTRE'])['VELOCIDAD_EFECTIVA_UPSTREAM']
fijo_velocidad_down, fijo_velocidad_up = [
    pd.concat([f.median().rename('median'), f.describe()[['min', '25%', '50%', '75%', 'max', 'mean', 'std']]], axis=1) for f in [fijo_velocidad_down, fijo_velocidad_up]
]

In [130]:
fijo_velocidad_down

median    min    25%    50%     75%        max        mean  \
ANNO TRIMESTRE                                                               
2017 2            5.00  0.009  2.000   5.00   10.00   204800.0   67.628981   
     3            5.00  0.009  2.000   5.00   10.00   204800.0   64.999430   
     4            5.00  0.009  2.000   5.00   10.00   204800.0   96.344426   
2018 1            5.12  0.009  2.048   5.12   10.00    50000.0   35.452908   
     2            6.00  0.009  2.500   6.00   10.24    30720.0   27.624643   
     3            6.00  0.009  3.000   6.00   12.00    50000.0   55.233938   
     4            6.00  0.009  3.000   6.00   14.00    50000.0   46.348453   
2019 1            6.00  0.009  3.000   6.00   15.00   122880.0   27.892722   
     2            8.00  0.009  4.000   8.00   20.00    50000.0   47.620466   
     3            8.00  0.009  3.000   8.00   20.00   409600.0   56.042497   
     4            9.00  0.009  4.000   9.00   20.00   409600.0   41.228925   
2020 1           10.00  0.003  4.000  10.00   20.00   716800.0   54.438478   
     2           10.00  0.003  4.000  10.00   22.00   512000.0  170.042358   
     3           10.00  0.003  4.100  10.00   25.00    50000.0   61.551479   
     4           10.00  0.003  5.000  10.00   30.00   100000.0   54.680532   
2021 1           10.00  0.003  5.000  10.00   30.00  2204800.0   83.557704   
     2           10.24  0.003  5.000  10.24   30.00  2204800.0   96.895239   
     3           12.00  0.010  5.120  12.00   40.00  2204800.0  123.710493   
     4           13.00  0.003  6.000  13.00   50.00  2204800.0  148.073308   
2022 1           15.00  0.003  6.000  15.00   50.00   310000.0   96.433428   
     2           20.00  0.000  6.000  20.00  100.00  3450300.0  190.391796   

                        std  
ANNO TRIMESTRE               
2017 2          1507.933459  
     3          1574.002825  
     4          1504.535982  
2018 1           491.471883  
     2           338.957006  
     3           776.193489  
     4           644.829783  
2019 1           571.099646  
     2           626.484777  
     3          1605.535365  
     4          1670.056647  
2020 1          2860.332336  
     2          3867.951949  
     3           682.308320  
     4           692.759296  
2021 1          7203.433775  
     2          7104.675474  
     3          8340.892500  
     4          9207.385039  
2022 1          1435.801755  
     2          9773.053848

In [146]:
fijo_velocidad_down.to_csv('data/internetfijo_velocidadefectiva_down.csv')

In [131]:
fijo_velocidad_up

median    min  25%   50%     75%        max        mean  \
ANNO TRIMESTRE                                                            
2017 2            1.00  0.009  0.8  1.00   2.000   204800.0   43.782694   
     3            1.00  0.019  1.0  1.00   2.048   204800.0   40.783730   
     4            1.00  0.008  1.0  1.00   2.500   204800.0   39.082905   
2018 1            1.00  0.008  1.0  1.00   3.000    20000.0   16.765971   
     2            1.00  0.019  1.0  1.00   3.000    20000.0   15.823991   
     3            1.00  0.019  1.0  1.00   4.000    40957.0   19.314962   
     4            1.00  0.008  1.0  1.00   4.000    20000.0   14.701498   
2019 1            1.00  0.008  1.0  1.00   4.000    55600.0   14.728370   
     2            1.02  0.008  1.0  1.02   5.000    24000.0   16.457708   
     3            1.50  0.008  1.0  1.50   5.000   409600.0   24.975379   
     4            2.00  0.009  1.0  2.00   5.000   409600.0   25.435757   
2020 1            2.00  0.002  1.0  2.00   5.000   716800.0   36.367277   
     2            2.10  0.002  1.0  2.10   7.000   512000.0  120.467532   
     3            2.50  0.002  1.0  2.50   7.000    44044.0   33.756212   
     4            3.00  0.002  1.0  3.00   8.000   100000.0   33.569093   
2021 1            3.00  0.002  1.0  3.00   8.000  2204800.0   62.641887   
     2            3.00  0.003  1.0  3.00  10.000  2204800.0   74.344228   
     3            4.00  0.010  1.0  4.00  10.000  2204800.0   96.539372   
     4            4.00  0.003  1.0  4.00  10.000  2204800.0  119.370526   
2022 1            5.00  0.003  1.0  5.00  12.610   310000.0   70.094706   
     2            6.00  0.000  2.0  6.00  20.000  3450300.0  146.290875   

                        std  
ANNO TRIMESTRE               
2017 2          1444.936110  
     3          1502.391186  
     4          1170.348889  
2018 1           220.457301  
     2           228.402898  
     3           314.737683  
     4           162.191675  
2019 1           310.757636  
     2           212.293043  
     3          1499.995346  
     4          1660.236201  
2020 1          2845.161039  
     2          3627.203825  
     3           618.434818  
     4           600.817432  
2021 1          7203.244610  
     2          7104.513966  
     3          8257.100882  
     4          9133.186780  
2022 1          1405.503956  
     2          9653.691414

In [147]:
fijo_velocidad_up.to_csv('data/internetfijo_velocidadefectiva_up.csv')